In [1]:
import sympy
from openLoop.utilities.ipynb.displays import *
from openLoop.utilities.ipynb.ipy_sympy import *
import scipy.linalg


import numpy.testing as np_test
import declarative

from test_SVD import SVD_gen_check, gen_rand_unitary
from openLoop.system import DAG_algorithm
from openLoop.system import SRE_matrix_algorithms
from openLoop.system import scisparse_algorithm

import timeit

asavefig.org_subfolder = 'plots'

Populating the interactive namespace from numpy and matplotlib
Sympy version:  1.0


In [2]:
from functools import reduce
def SVD_compare_error(
    N = 10,
    length = 10,
    solver =  DAG_algorithm,
    N_in = None,
    N_out = None,
):
    U = gen_rand_unitary(N = N, length = length)
    V = gen_rand_unitary(N = N, length = length)

    seq = dict()
    req = dict()
    edge_map = dict()
    S_diags = []
    for idx in range(N):
        s_diag = 10**(-5 + 10 * np.random.random(length))
        edge_map[idx, idx] = s_diag
        S_diags.append(s_diag)
        seq[idx] = set([idx])
        req[idx] = set([idx])
    S = seq, req, edge_map
    condition = reduce(np.maximum, S_diags) / reduce(np.minimum, S_diags)

    M = SRE_matrix_algorithms.matrix_mult_sre(
        SRE_matrix_algorithms.matrix_mult_sre(U, S), V
    )

    SRE_matrix_algorithms.check_sre(M)
    sparsity = SRE_matrix_algorithms.SRE_count_sparsity(M)
    print("SPARSITY FRAC: ", sparsity)

    if N_in is None:
        inputs_set = set(range(N))
    else:
        inputs_set = set(range(N-N_in, N))
    if N_out is None:
        outputs_set = set(range(N))
    else:
        mid = N_out // 2
        outputs_set = set(range(0, mid)) | set(range(N-(N_out - mid), N))

    Mseq, Mreq, Medge_map = SRE_matrix_algorithms.copy_sre(M)
    print(solver)
    time_start = timeit.default_timer()
    sbunch = solver.inverse_solve_inplace(
        seq = Mseq,
        req = Mreq,
        edge_map = Medge_map,
        inputs_set = inputs_set,
        outputs_set = outputs_set,
        verbose = True,
        negative = False,
    )
    time_end = timeit.default_timer()
    b = declarative.Bunch( 
        time = time_end - time_start,
        length = length,
    )
    b.update(sparsity)
    return b

In [3]:
mats = []
for N in [10, 30, 100, 300]:
    for length in [10, 100, 1000, 10000]:
        for inst in range(3):
            r = SVD_compare_error(
                N = N,
                length = length,
                N_in = 1,
            )
            mats.append(r)

SPARSITY FRAC:  {'Nnodes': 10, 'Nedges': 20, 'density_sq': 0.2, 'density_lin': 2.0}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 10
TRIVIAL STAGE, REMAINING 9
BADGUY STAGE, REMAINING 9
SPARSITY FRAC:  {'Nnodes': 10, 'Nedges': 32, 'density_sq': 0.32, 'density_lin': 3.2}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 10
TRIVIAL STAGE, REMAINING 9
BADGUY STAGE, REMAINING 9
SPARSITY FRAC:  {'Nnodes': 10, 'Nedges': 26, 'density_sq': 0.26, 'density_lin': 2.6}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 10
TRIVIAL STAGE, REMAINING 10
BADGUY STAGE, REMAINING 10
SPARSITY FRAC:  {'Nnodes': 10, 'Nedges': 49, 'density_sq': 0.49, 'density_lin': 4.9}
<module 'openLoop.system.D

/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py:645: RuntimeWarning: invalid value encountered in true_divide
  u_mod_edge = edge + norm_orig * edge / abs(edge)
/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py:365: RuntimeWarning: invalid value encountered in greater
  rel_r_to_c = np.asarray(np.count_nonzero(normr > normc))
/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py:387: RuntimeWarning: invalid value encountered in greater_equal
  bignodes_r = np.array(rvec) >= 1./(len(req[node]))**.5
/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py:950: RuntimeWarning: divide by zero encountered in true_divide
  CLG = -1 / self_edge
/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py:950: RuntimeWarning: invalid value encountered in true_divide
  CLG = -1 / self_edge
/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG

SPARSITY FRAC:  {'Nnodes': 100, 'Nedges': 247, 'density_sq': 0.0247, 'density_lin': 2.47}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 100
TRIVIAL STAGE, REMAINING 100
BADGUY STAGE, REMAINING 100
SPARSITY FRAC:  {'Nnodes': 100, 'Nedges': 318, 'density_sq': 0.0318, 'density_lin': 3.18}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 100
TRIVIAL STAGE, REMAINING 100
BADGUY STAGE, REMAINING 100
SPARSITY FRAC:  {'Nnodes': 300, 'Nedges': 960, 'density_sq': 0.010666666666666666, 'density_lin': 3.2}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 300
TRIVIAL STAGE, REMAINING 300
BADGUY STAGE, REMAINING 300
SPARSITY FRAC:  {'Nnodes': 300, 'Nedges': 1035, 'density_sq': 0.0115,

In [4]:
axB = mplfigB(Nrows=1)
color_by_len = {
    10 : 'blue',
    100 : 'green',
    1000 : 'purple',
    10000 : 'red',
}
for r in mats:
    axB.ax0.scatter(
        r.Nnodes, 
        r.time,
        color = color_by_len[r.length],
    )
axB.ax0.set_xscale('log')
axB.ax0.set_yscale('log')
axB.ax0.set_xlim(9, 400)
axB.save('timing_snode_direct')

figure: plots/timing_snode_direct.png
[[file:plots/timing_snode_direct.png]]


In [5]:
axB = mplfigB(Nrows=1)
for r in mats:
    axB.ax0.scatter(
        r.Nnodes, 
        r.time / r.Nnodes,
        color = color_by_len[r.length],
    )
axB.ax0.set_xscale('log')
axB.ax0.set_yscale('log')
axB.ax0.set_xlim(9, 400)
axB.save('timing_snode_relsq2')

figure: plots/timing_snode_relsq2.png
[[file:plots/timing_snode_relsq2.png]]


In [6]:
axB = mplfigB(Nrows=1)
for r in mats:
    axB.ax0.scatter(
        r.Nnodes, 
        r.time / r.length,
        color = color_by_len[r.length],
    )
axB.ax0.set_xscale('log')
axB.ax0.set_yscale('log')
axB.ax0.set_xlim(9, 400)
axB.save('timing_snode_rellength')

figure: plots/timing_snode_rellength.png
[[file:plots/timing_snode_rellength.png]]
